In [1]:
using Compat
using Docile
using Iterators
using Pipe

macro printval(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr," = ", $ee)))
end

macro pz(ee)
    ee_expr = @sprintf "%s" string(ee)
    esc(:(println($ee_expr,"\t\t",typeof($ee), "\t", size($ee))))
end

In [101]:
function unzip(xs)
    [zip(xs...)...]
end

unzip (generic function with 1 method)

In [2]:
push!(LOAD_PATH, map(x->"../"*x, filter(fn-> !(contains(fn,".")),readdir("..")))...)


11-element Array{Union(ASCIIString,UTF8String),1}:
 "/root/buildFromSource/julia/usr/local/share/julia/site/v0.3"
 "/root/buildFromSource/julia/usr/share/julia/site/v0.3"      
 "../Corpus"                                                  
 "../doc2vec"                                                 
 "../Models"                                                  
 "../Optimisation"                                            
 "../recursive_embeddings"                                    
 "../summaristation"                                          
 "../tools"                                                   
 "../util"                                                    
 "../word-embedding3"                                         

In [3]:
using WordEmbeddings

In [4]:
training = open("../Corpus/serialised/opinosis_train_dev_plain.jsz","r") do fs
    deserialize(fs)
end
@pz training

training		Array{Array{String,1},1}	(6097,)


In [64]:

function pad{S<:String}(sent::Vector{S}, padded_length, pad_word="*STARTPAD*")
    if length(sent) <= padded_length
        ret =  fill(pad_word,padded_length)
        ret[end-length(sent)+1:end] = sent
        ret
    else
        sent
    end
end

pad (generic function with 2 methods)

In [58]:

import WordEmbeddings.WE
function WE(N::DataType,S::DataType, embedding_width::Int)
    L=Array(N,(embedding_width,0))
    word_index=Dict{S,Int}()
    indexed_words=S[]
    WE(L,word_index,indexed_words)
end



@doc "Gets the word index, or creates one if it doesn't already exist" ->
function get_word_index!{N,S, S2}(we::WE{N,S}, word::S2, word_varience = 0.01)
    if (word in keys(we.word_index))
        we.word_index[word]
    else
        index = length(we.indexed_words)+1
        we.word_index[word]=index
        push!(we.indexed_words,word)
        
        embedding = convert(Vector{N},word_varience.*randn(size(we.L,1)))
        we.L = hcat(we.L,embedding)
        index
    end
end

function add_all_words!{N,S}(we::WE{N,S}, words::Vector{S}, word_varience=0.01)
    for word in words
        get_word_index!(we, word, word_varience)
    end
    we
end
function add_all_words!{N,S}(we::WE{N,S}, paras::Vector{Vector{S}}, word_varience=0.01)
    for para in paras
        add_all_words!(we, para, word_varience)
    end
    we
end

add_all_words! (generic function with 4 methods)

In [59]:
type PVDM{N<:Number, S<:AbstractString}
    we::WE
    
    pe::WE #use a word embedder for Paragraphs too
    
    
    W::AbstractMatrix{N}
    b::AbstractVector{N}

    window_length::Int
    varience::N
end

function PVDM{N,S}(we::WE{N,S}, window_length::Int, varience=0.001)
    
    emb_width,n_words = size(we.L)
    concat_layer_width = emb_width*(window_length+1)
    const W = convert(Matrix{N}, varience*randn(n_words,concat_layer_width))
    const b = convert(Vector{N}, varience*randn(n_words))
    
    pe = WE(N,Vector{S},emb_width)
    
    PVDM{N,S}( we, pe, W,b, window_length, varience)
end
    
    

LoadError: invalid redefinition of constant PVDM
while loading In[59], in expression starting on line 1

In [65]:
const WINDOW_LEN = 8 
training = Vector{String}[pad(para, WINDOW_LEN+1) for para in training]

we_outer = WE(Float32,String, 200)
add_all_words!(we_outer, training)
pvdm = PVDM(we_outer, WINDOW_LEN);
we_outer=0

0

In [42]:
@doc """gets the training cases as vector of (paraIndex, [word_indexes], label_word_index),
cycling by the window length.
Adds the paragraph if it does not already have an index
""" ->
function get_para_training_cases!{S<:String}(pvdm::PVDM, para::Vector{S})
    para_ind = get_word_index!(pvdm.pe, para)
    
    Task() do 
        @assert length(para)>=pvdm.window_length+1
        for offset in 0:length(para)-(pvdm.window_length+1)
            window_iis = [1:pvdm.window_length;]+offset
            label_ii = pvdm.window_length+1+offset
            
            window_words = para[window_iis]
            label_word = para[label_ii]
                        
            windows_indexes = map(word->get_word_index(pvdm.we, word), window_words)
            label_index = get_word_index(pvdm.we, label_word)
            
            produce(para_ind, windows_indexes, label_index)
        end
    end
    
end

get_para_training_cases! (generic function with 2 methods)

In [97]:
training_indexes = chain(map(para -> get_para_training_cases!(pvdm, para), paddded_training)...) |> collect



73472-element Array{Any,1}:
 (1,[1,2,3,4,5,6,7,8],9)                    
 (1,[2,3,4,5,6,7,8,9],10)                   
 (2,[11,12,13,14,6,15,3,16],5)              
 (2,[12,13,14,6,15,3,16,5],6)               
 (2,[13,14,6,15,3,16,5,6],17)               
 (2,[14,6,15,3,16,5,6,17],5)                
 (2,[6,15,3,16,5,6,17,5],18)                
 (2,[15,3,16,5,6,17,5,18],19)               
 (2,[3,16,5,6,17,5,18,19],20)               
 (2,[16,5,6,17,5,18,19,20],17)              
 (2,[5,6,17,5,18,19,20,17],5)               
 (2,[6,17,5,18,19,20,17,5],6)               
 (2,[17,5,18,19,20,17,5,6],21)              
 ⋮                                          
 (5642,[2956,50,6,1616,14,1069,35,3],116)   
 (5642,[50,6,1616,14,1069,35,3,116],1999)   
 (5642,[6,1616,14,1069,35,3,116,1999],32)   
 (5642,[1616,14,1069,35,3,116,1999,32],1216)
 (5642,[14,1069,35,3,116,1999,32,1216],910) 
 (5642,[1069,35,3,116,1999,32,1216,910],3)  
 (5642,[35,3,116,1999,32,1216,910,3],50)    
 (5642,[3,116,1999,32,1216,

In [112]:
function get_input_layer(pvdm::PVDM, para_index::Int, window_indexes::Vector{Int})
    [pvdm.pe.L[:,para_index], vec(pvdm.we.L[:,window_indexes])]
end

get_input_layer (generic function with 1 method)

In [113]:
get_input_layer(pvdm, training_indexes[1][1], training_indexes[1][2])

1800-element Array{Float32,1}:
  0.0107726  
 -0.000616014
 -0.00549025 
  0.0184816  
  0.000684042
 -0.00161514 
 -0.0165534  
  0.00620565 
  0.00690872 
  0.0222866  
 -0.0144855  
  0.00505583 
  0.0152064  
  ⋮          
  0.00498716 
 -0.00197576 
 -0.019022   
 -0.0138082  
  0.00210936 
  0.00489302 
 -0.00920056 
 -0.0124299  
 -0.0106149  
 -0.00313912 
 -0.00327577 
  0.00329835 